#라이브러리 및 데이터 임포트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/데이터셋/2_train test/21년_22년_567/train_data.csv')
train['시간'] = pd.to_datetime(train['시간'])
train

,대여소번호,시간,월,요일,시,대여횟수,반납횟수,순대여량
0,102,2021-05-01 00:00:00,5,5,0,0.0,1.0,-1.0
1,102,2021-05-01 01:00:00,5,5,1,2.0,2.0,0.0
2,102,2021-05-01 02:00:00,5,5,2,1.0,0.0,1.0
3,102,2021-05-01 03:00:00,5,5,3,1.0,0.0,1.0
4,102,2021-05-01 04:00:00,5,5,4,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
5131387,4863,2021-07-31 19:00:00,7,5,19,0.0,0.0,0.0
5131388,4863,2021-07-31 20:00:00,7,5,20,0.0,0.0,0.0
5131389,4863,2021-07-31 21:00:00,7,5,21,2.0,0.0,2.0
5131390,4863,2021-07-31 22:00:00,7,5,22,0.0,1.0,-1.0


In [ ]:
train['이용량'] = train['대여횟수'] + train['반납횟수']
train

,대여소번호,시간,월,요일,시,대여횟수,반납횟수,순대여량,이용량
0,102,2021-05-01 00:00:00,5,5,0,0.0,1.0,-1.0,1.0
1,102,2021-05-01 01:00:00,5,5,1,2.0,2.0,0.0,4.0
2,102,2021-05-01 02:00:00,5,5,2,1.0,0.0,1.0,1.0
3,102,2021-05-01 03:00:00,5,5,3,1.0,0.0,1.0,1.0
4,102,2021-05-01 04:00:00,5,5,4,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5131387,4863,2021-07-31 19:00:00,7,5,19,0.0,0.0,0.0,0.0
5131388,4863,2021-07-31 20:00:00,7,5,20,0.0,0.0,0.0,0.0
5131389,4863,2021-07-31 21:00:00,7,5,21,2.0,0.0,2.0,2.0
5131390,4863,2021-07-31 22:00:00,7,5,22,0.0,1.0,-1.0,1.0


#업무지구/생활지구 분리

##평일 출퇴근 시간에 따라 구분
출퇴근시간: 07시-09시, 17시-19시
<br>

In [2]:
train["workhour"] = np.where(((train["시"] >= 17) & (train["시"] <= 19)) | ((train["시"] >= 7) & (train["시"] <= 9)) , 1, 0)
train = train[train['요일']<5]

NameError: ignored

In [ ]:
df = pd.DataFrame(train.groupby(['대여소번호', 'workhour'])['이용량'].mean()).reset_index()
df.columns = ['대여소번호', 'workhour', 'mean']
df['std'] = list(train.groupby(['대여소번호', 'workhour'])['이용량'].std())
df.columns = ['대여소번호', 'workhour', 'mean', 'std']
df

##출퇴근시간과 그외시간의 평균이용량에 유의한 차이가 있는지 t-test
유의수준 5%

In [ ]:
mean_diff = []
std_avg = []

for i in df['대여소번호'].unique():
  mean_1 = df[(df['대여소번호']==i)&(df['workhour']==1)]['mean'].values[0]
  mean_0 = df[(df['대여소번호']==i)&(df['workhour']==0)]['mean'].values[0]
  mean_diff.append(mean_1-mean_0)
  std_1 = df[(df['대여소번호']==i)&(df['workhour']==1)]['std'].values[0]
  std_0 = df[(df['대여소번호']==i)&(df['workhour']==0)]['std'].values[0]
  std_avg.append(np.sqrt((std_1**2/6)-(std_0**2/18)))


In [3]:
df_for_split = pd.DataFrame({'대여소번호':df['대여소번호'].unique(), '평균차이':mean_diff, '표준오차':std_avg})
df_for_split['t_stats'] = df_for_split['평균차이'] / df_for_split['표준오차']
df_for_split['p_value'] = 1-scipy.stats.norm.cdf(df_for_split['t_stats'])
df_for_split

NameError: ignored

In [4]:
temp = pd.read_csv('/content/drive/MyDrive/데이터셋/0_raw_data/raw_data/공공자전거 대여이력 정보_2021.05.csv', encoding = 'cp949')
temp

NameError: ignored

In [5]:
temp1 = temp.iloc[:, 2:4]
temp1 = temp1.drop_duplicates(subset=['대여 대여소번호', '대여 대여소명'])
temp1.columns = ['대여소번호', '대여소명']
temp1

NameError: ignored

In [ ]:
temp2 = pd.merge(df_for_split, temp1, how = 'left', left_on = '대여소번호', right_on = '대여소번호')
temp2 = temp2[['대여소명',	'평균차이',	'표준오차',	't_stats',	'p_value']]
temp2

In [ ]:
work_list = df_for_split[df_for_split['p_value']<=0.05]['대여소번호'].unique()
live_list = df_for_split[df_for_split['p_value']>0.05]['대여소번호'].unique()

#클러스터링

##대여소별 위치

In [1]:
locations = pd.read_csv('/content/drive/MyDrive/데이터셋/3_live_work_and_clustering/version_1/locations.csv', encoding = 'cp949')
locations = locations.iloc[:, :6]
locations.columns = ['ID', 'Name', 'Gu', 'Juso', 'latitude', 'longitude']
locations = locations.dropna()
locations = locations.sort_values('ID').reset_index(drop = True)

NameError: ignored

In [ ]:
plt.figure(figsize = (16, 10))
plt.scatter(locations['longitude'], locations['latitude'])
plt.show()

In [ ]:
locations_work = locations[locations['ID'].isin(work_list)==True]
locations_live = locations[locations['ID'].isin(live_list)==True]

##업무지구

###스케일링

In [ ]:
temp = locations_work[['latitude', 'longitude']]
temp

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
locations_work_scaled = scaler.fit_transform(temp)
locations_work_scaled = pd.DataFrame(locations_work_scaled, columns = temp.columns, index = list(locations_work['ID']))
locations_work_scaled

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(locations_work_scaled['latitude'],locations_work_scaled['longitude'], '.')

###K-means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings

In [ ]:
warnings.filterwarnings('ignore')
for k in range(100, 601,100):
  score = []
  # kmeans는 랜덤 배정되는 초기 중심값에 따라 실루엣 계수가 달라지므로 여러번(여기선 10번)
  # 수행하여 평균값을 비교하도록 한다.
  for i in range(10):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(locations_work_scaled) # caution
    score.append(silhouette_score(locations_work_scaled, kmeans.labels_)) # caution
  avg = np.mean(score)
  print(f"k = {k} => %a " %avg)

In [ ]:
warnings.filterwarnings('ignore')
for k in range(100, 601,100):
  score = []
  # kmeans는 랜덤 배정되는 초기 중심값에 따라 실루엣 계수가 달라지므로 여러번(여기선 10번)
  # 수행하여 평균값을 비교하도록 한다.
  for i in range(10):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(locations_live_scaled) # caution
    score.append(silhouette_score(locations_live_scaled, kmeans.labels_)) # caution
  avg = np.mean(score)
  print(f"k = {k} => %a " %avg)

In [ ]:
km_work = KMeans(n_clusters = 200)

cluster_km_work = km_work.fit_predict(locations_work_scaled)
locations_work_km = locations_work_scaled
locations_work_km['cluster'] = cluster_km_work
locations_work_km

In [ ]:
plt.figure(figsize=(15, 12))
# sns.scatterplot(x = 'longitude', y = 'latitude', data = locations_work_km, hue = 'cluster', legend = False)
plt.scatter(locations_work_km['longitude'], locations_work_km['latitude'], c = locations_work_km['cluster'], cmap = 'plasma')
plt.title('K-means Clustering Result')
plt.show()

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics.cluster import silhouette_score
coef_km = metrics.silhouette_score(locations_work_scaled, cluster_km_work)
coef_km

In [ ]:
locations_work_km.iloc[:, :2] = scaler.inverse_transform(locations_work_scaled.iloc[:,:2])
locations_work_km

In [ ]:
variance_km_work = pd.DataFrame(np.sqrt(locations_work_km.groupby('cluster')['latitude', 'longitude'].var())).reset_index()
variance_km_work['latitude'] = variance_km_work['latitude'] * 111
variance_km_work['longitude'] = variance_km_work['longitude'] * 88.8
variance_km_work

In [ ]:
# plt.figure(figsize = (6,4))
# plt.subplot(1,2,1)
# plt.hist(variance_km_work['latitude'])
# plt.subplot(1,2,2)
# plt.hist(variance_km_work['longitude'])
# plt.show()

fig = plt.figure(figsize=(6, 4))
ax1 = fig.add_subplot(1, 2, 1)
ax1.hist(variance_km_work['latitude'], color = 'skyblue')
ax1.set_title('Var of latitude')
ax1.set_xlabel('Variance')
ax1.set_ylabel('Freq')

ax2 = fig.add_subplot(1, 2, 2)
ax2.hist(variance_km_work['longitude'], color = 'salmon')
ax2.set_title('Var of longitude')
ax2.set_xlabel('Variance')
ax2.set_ylabel('Freq')
plt.subplots_adjust(wspace = 0.5)
plt.show()

In [ ]:
print(variance_km_work['latitude'].mean())
print(variance_km_work['longitude'].mean())

In [ ]:
locations_work_km['ID'] = locations_work_km.index
locations_work_km.to_csv('/content/drive/MyDrive/데이터셋/3_live_work_and_clustering/version_2/clusters_for_work', index = False)

##거주지구

###스케일링

In [ ]:
temp = locations_live[['latitude', 'longitude']]
temp

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
locations_live_scaled = scaler.fit_transform(temp)
locations_live_scaled = pd.DataFrame(locations_live_scaled, columns = temp.columns, index = list(locations_live['ID']))
locations_live_scaled

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(locations_live_scaled['latitude'],locations_live_scaled['longitude'], '.')

###K-means Clustering

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
km_live = KMeans(n_clusters = 200)

cluster_km_live = km_live.fit_predict(locations_live_scaled)
locations_live_km = locations_live_scaled
locations_live_km['cluster'] = cluster_km_live
locations_live_km

In [ ]:
plt.figure(figsize=(15, 12))
# sns.scatterplot(x = 'longitude', y = 'latitude', data = locations_work_km, hue = 'cluster', legend = False)
plt.scatter(locations_live_km['longitude'], locations_live_km['latitude'], c = locations_live_km['cluster'], cmap = 'plasma')
plt.title('K-means Clustering Result of Living')
plt.show()

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics.cluster import silhouette_score
coef_km = metrics.silhouette_score(locations_live_scaled, cluster_km_live)
coef_km

In [ ]:
locations_live_km.iloc[:, :2] = scaler.inverse_transform(locations_live_scaled.iloc[:,:2])
locations_live_km

In [ ]:
variance_km_live = pd.DataFrame(np.sqrt(locations_live_km.groupby('cluster')['latitude', 'longitude'].var())).reset_index()
variance_km_live['latitude'] = variance_km_live['latitude'] * 111
variance_km_live['longitude'] = variance_km_live['longitude'] * 88.8
variance_km_live

In [ ]:
# plt.figure(figsize = (6,4))
# plt.subplot(1,2,1)
# plt.hist(variance_km_work['latitude'])
# plt.subplot(1,2,2)
# plt.hist(variance_km_work['longitude'])
# plt.show()

fig = plt.figure(figsize=(6, 4))
ax1 = fig.add_subplot(1, 2, 1)
ax1.hist(variance_km_live['latitude'], color = 'skyblue')
ax1.set_title('Var of latitude')
ax1.set_xlabel('Variance')
ax1.set_ylabel('Freq')

ax2 = fig.add_subplot(1, 2, 2)
ax2.hist(variance_km_live['longitude'], color = 'salmon')
ax2.set_title('Var of longitude')
ax2.set_xlabel('Variance')
ax2.set_ylabel('Freq')
plt.subplots_adjust(wspace = 0.5)
plt.show()

In [ ]:
print(variance_km_live['latitude'].mean())
print(variance_km_live['longitude'].mean())

In [ ]:
#locations_live_km['ID'] = locations_live_km.index

In [ ]:
locations_live_km['cluster'] = locations_live_km['cluster'] + 200
locations_live_km

In [ ]:
locations_live_km.to_csv('/content/drive/MyDrive/데이터셋/3_live_work_and_clustering/version_2/clusters_for_live', index = False)

#클러스터 결과를 원 데이터에 merge

##train set

In [ ]:
cluster_work = locations_work_km.iloc[:,2:].reset_index(drop=True)
cluster_live = locations_live_km.iloc[:,2:].reset_index(drop=True)
cluster_total = pd.concat([cluster_work, cluster_live])
cluster_total = cluster_total.sort_values('cluster')
cluster_total

,cluster,ID
786,0,2412.0
959,0,3623.0
789,0,2420.0
790,0,2423.0
777,0,2342.0
...,...,...
154,399,NaN
153,399,NaN
150,399,NaN
164,399,NaN


In [ ]:
train_clustered = train.copy()
train_clustered = train_clustered[train_clustered['대여소번호'].isin(cluster_total['ID'])]
train_clustered

,대여소번호,시간,월,요일,시,대여횟수,반납횟수,순대여량,이용량,workhour
48,102,2021-05-03 00:00:00,5,0,0,0.0,0.0,0.0,0.0,0
49,102,2021-05-03 01:00:00,5,0,1,0.0,0.0,0.0,0.0,0
50,102,2021-05-03 02:00:00,5,0,2,2.0,0.0,2.0,2.0,0
51,102,2021-05-03 03:00:00,5,0,3,0.0,0.0,0.0,0.0,0
52,102,2021-05-03 04:00:00,5,0,4,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
5126947,4860,2021-07-30 19:00:00,7,4,19,0.0,1.0,-1.0,1.0,1
5126948,4860,2021-07-30 20:00:00,7,4,20,0.0,3.0,-3.0,3.0,0
5126949,4860,2021-07-30 21:00:00,7,4,21,3.0,1.0,2.0,4.0,0
5126950,4860,2021-07-30 22:00:00,7,4,22,2.0,1.0,1.0,3.0,0


In [ ]:
print(len(train_clustered['대여소번호'].unique()))
print(len(train_clustered['대여소번호'].unique()) * len(train_clustered['시간'].unique()))

1057
1648920


In [ ]:
train_clustered = pd.merge(train_clustered, cluster_total, how = 'left', left_on = '대여소번호', right_on = 'ID')
train_clustered = train_clustered[['cluster', '시간', '월', '요일', '시', '대여횟수', '반납횟수', '순대여량']]
train_clustered

,cluster,시간,월,요일,시,대여횟수,반납횟수,순대여량
0,87,2021-05-03 00:00:00,5,0,0,0.0,0.0,0.0
1,87,2021-05-03 01:00:00,5,0,1,0.0,0.0,0.0
2,87,2021-05-03 02:00:00,5,0,2,2.0,0.0,2.0
3,87,2021-05-03 03:00:00,5,0,3,0.0,0.0,0.0
4,87,2021-05-03 04:00:00,5,0,4,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1648915,50,2021-07-30 19:00:00,7,4,19,0.0,1.0,-1.0
1648916,50,2021-07-30 20:00:00,7,4,20,0.0,3.0,-3.0
1648917,50,2021-07-30 21:00:00,7,4,21,3.0,1.0,2.0
1648918,50,2021-07-30 22:00:00,7,4,22,2.0,1.0,1.0


In [ ]:
train_clustered_final = pd.DataFrame(train_clustered.groupby(['cluster', '시간'])['순대여량'].sum()).reset_index()
train_clustered_final

,cluster,시간,순대여량
0,0,2021-05-03 00:00:00,2.0
1,0,2021-05-03 01:00:00,-1.0
2,0,2021-05-03 02:00:00,0.0
3,0,2021-05-03 03:00:00,0.0
4,0,2021-05-03 04:00:00,-1.0
...,...,...,...
311995,199,2021-07-30 19:00:00,6.0
311996,199,2021-07-30 20:00:00,-2.0
311997,199,2021-07-30 21:00:00,3.0
311998,199,2021-07-30 22:00:00,5.0


In [ ]:
400 * len(train_clustered['시간'].unique())

624000

In [ ]:
# train_clustered_final.to_csv('/content/drive/MyDrive/데이터셋/clustering/train_clustered.csv', index = False)

##test set

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/데이터셋/train test/21년_22년_567/test_data.csv')
test['시간'] = pd.to_datetime(test['시간'])

FileNotFoundError: ignored

In [ ]:
test_clustered = test.copy()
test_clustered = test_clustered[test_clustered['대여소번호'].isin(cluster_total['ID'])]
test_clustered

In [ ]:
print(len(test_clustered['대여소번호'].unique()))
print(len(test_clustered['대여소번호'].unique()) * len(test_clustered['시간'].unique()))

In [ ]:
test_clustered = pd.merge(test_clustered, cluster_total, how = 'left', left_on = '대여소번호', right_on = 'ID')
test_clustered = test_clustered[['cluster', '시간', '월', '요일', '시', '대여횟수', '반납횟수', '순대여량']]
test_clustered

In [ ]:
test_clustered_final = pd.DataFrame(test_clustered.groupby(['cluster', '시간'])['순대여량'].sum()).reset_index()
test_clustered_final

In [ ]:
400 * len(test_clustered['시간'].unique())

In [ ]:
# test_clustered_final.to_csv('/content/drive/MyDrive/데이터셋/clustering/test_clustered.csv', index = False)

# 과반납/과대여 시각화

In [ ]:
train_clustered = train.copy()
train_clustered = train_clustered[train_clustered['대여소번호'].isin(cluster_total['ID'])]
train_clustered

,대여소번호,시간,월,요일,시,대여횟수,반납횟수,순대여량,이용량,workhour
48,102,2021-05-03 00:00:00,5,0,0,0.0,0.0,0.0,0.0,0
49,102,2021-05-03 01:00:00,5,0,1,0.0,0.0,0.0,0.0,0
50,102,2021-05-03 02:00:00,5,0,2,2.0,0.0,2.0,2.0,0
51,102,2021-05-03 03:00:00,5,0,3,0.0,0.0,0.0,0.0,0
52,102,2021-05-03 04:00:00,5,0,4,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
5126947,4860,2021-07-30 19:00:00,7,4,19,0.0,1.0,-1.0,1.0,1
5126948,4860,2021-07-30 20:00:00,7,4,20,0.0,3.0,-3.0,3.0,0
5126949,4860,2021-07-30 21:00:00,7,4,21,3.0,1.0,2.0,4.0,0
5126950,4860,2021-07-30 22:00:00,7,4,22,2.0,1.0,1.0,3.0,0


In [ ]:
# locations_work = locations_work.drop(['Juso'], axis = 1)
locations_work.columns = ['ID', 'latitude', 'longitude']
df = pd.merge(train_clustered, locations_work, left_on = '대여소번호', right_on = 'ID').drop('ID', axis = 1)

In [ ]:
plt.figure(figsize = (16, 10))
plt.scatter(df['longitude'], df['latitude'])

In [ ]:
df = df.loc[(df['longitude']>126.882) & (df['longitude'] < 126.982) & (df['latitude'] > 37.499) & (df['latitude'] < 37.55) & (df['workhour'] == 1) & (df['시'] < 10)]
df_loc = pd.merge(df.groupby('대여소번호').sum()[['순대여량']].reset_index(), 
         df[['대여소번호', 'latitude', 'longitude']].drop_duplicates(), 
         on = '대여소번호')

In [ ]:
m = folium.Map(location=[37.523, 126.932], tiles="OpenStreetMap", zoom_start=14)


# 과대여 -> 파란색 // 과반납 -> 빨간색
for i in range(len(df_loc)):
    color = 'blue' if df_loc.iloc[i]['순대여량'] >= 0 else 'red'  
    folium.CircleMarker(
        location=[df_loc.iloc[i]['latitude'], df_loc.iloc[i]['longitude']],
        radius=abs(df_loc.iloc[i]['순대여량']) / 10 + 1,
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)


In [ ]:
# 마커(지도 범위 탐색)
# folium.Marker([37.523, 126.932], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.499, 126.932], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.549, 126.932], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.523, 126.982], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.523, 126.882], popup="Seoul", tooltip="Seoul").add_to(m)
# 범례 달기
legend_html = """
<div style="position: fixed; bottom: 50px; right: 50px; width: 160px; height: 75px; border:4px solid black; z-index:9999; font-size:20px;; font-weight:bold">
&nbsp; <i class="fa fa-circle" style="color:blue"> &nbsp; 과대여 </i><br>
&nbsp; <i class="fa fa-circle" style="color:red"> &nbsp; 과반납 </i>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))
m

In [ ]:
# locations_work = locations_work.drop(['Juso'], axis = 1)
locations_work.columns = ['ID', 'latitude', 'longitude']
df = pd.merge(train_clustered, locations_work, left_on = '대여소번호', right_on = 'ID').drop('ID', axis = 1)

In [ ]:
# 퇴근 시간대, 여의도
df = df.loc[(df['longitude']>126.882) & (df['longitude'] < 126.982) & (df['latitude'] > 37.499) & (df['latitude'] < 37.55) & (df['workhour'] == 1) & (df['시'] >12)]
df_loc = pd.merge(df.groupby('대여소번호').sum()[['순대여량']].reset_index(), 
         df[['대여소번호', 'latitude', 'longitude']].drop_duplicates(), 
         on = '대여소번호')

In [ ]:
m = folium.Map(location=[37.523, 126.932], tiles="OpenStreetMap", zoom_start=14)


# 과대여 -> 파란색 // 과반납 -> 빨간색
for i in range(len(df_loc)):
    color = 'blue' if df_loc.iloc[i]['순대여량'] >= 0 else 'red'  
    folium.CircleMarker(
        location=[df_loc.iloc[i]['latitude'], df_loc.iloc[i]['longitude']],
        radius=abs(df_loc.iloc[i]['순대여량']) / 10 + 1,
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)

# 마커(지도 범위 탐색)
# folium.Marker([37.523, 126.932], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.499, 126.932], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.549, 126.932], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.523, 126.982], popup="Seoul", tooltip="Seoul").add_to(m)
# folium.Marker([37.523, 126.882], popup="Seoul", tooltip="Seoul").add_to(m)
# 범례 달기
legend_html = """
<div style="position: fixed; bottom: 50px; right: 50px; width: 160px; height: 75px; border:4px solid black; z-index:9999; font-size:20px;; font-weight:bold">
&nbsp; <i class="fa fa-circle" style="color:blue"> &nbsp; 과대여 </i><br>
&nbsp; <i class="fa fa-circle" style="color:red"> &nbsp; 과반납 </i>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))
m